# Database Restore

**Objective:** Restore a database from CSV files in /data/backup/.

In [1]:
import os, sys, time
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if '__' not in x and len(x)>8])

['DataframeToDicts', 'FigureToJson', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import django.db.models as JM 
import database as DB
print([x for x in dir(DB) if x[0] != '_'])

['BaseManager', 'DeleteRecord', 'DeleteTable', 'GetNativeTableCounts', 'GetRecord', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle', 'MD', 'time']


In [5]:
DATA_PATH = os.path.join(MODULE_PATH, 'data')
os.path.exists(DATA_PATH)

True

In [6]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import IPython.display as DS 
from ipywidgets import IntProgress

### Restore

In [7]:
backupPath = os.path.join(DATA_PATH, 'backup')
moduleLs = [m.name for m in os.scandir(backupPath) if m.is_dir()]

for m in moduleLs:
    modulePath = os.path.join(backupPath, m)
    tableLs = [t.name for t in os.scandir(modulePath) if t.is_dir()]
        
    for t in tableLs:
        tablePath = os.path.join(modulePath, t)
        fileLs = [f.name for f in os.scandir(tablePath) if not f.is_dir()]
        
        print(m, t, len(fileLs))
        progressBar = IntProgress(min=0, max=len(fileLs) or 1) 
        DS.display(progressBar) 

        if len(fileLs) == 0:
            progressBar.value += 1

        for f in fileLs:
            filePath = os.path.join(tablePath, f)
            blockDf = PD.read_csv(filePath)
            blockLs = NT.DataframeToDicts(blockDf)
            DB.InsertBulk(m, t, blockLs)
            progressBar.value += 1
            #time.sleep(1)

members User 0


IntProgress(value=0, max=1)

movies Load_InternetMDB 6


IntProgress(value=0, max=6)

movies Load_MovieDB 28


...

IntProgress(value=0, max=1)

movies StreamingMovie 3


IntProgress(value=0, max=3)

movies StreamingService 1


IntProgress(value=0, max=1)

In [8]:
summaryLs = DB.GetTableCounts()
PD.DataFrame(summaryLs)

,Module,Table,Count
0,members,User,1
1,movies,MasterMovie,6826
2,movies,Load_MovieDB,27417
3,movies,Load_InternetMDB,5488
4,movies,Load_RottenTomatoes,11701
5,movies,DataChanges,6
6,movies,StreamingService,71
7,movies,StreamingMovie,2012
8,central,UserProfile,1
9,central,VotedStars,28


In [9]:
#DB.DeleteTable('movies', 'StreamingService')

### Check Results

In [10]:
mainLs = DB.GetTableDictionary('movies', 'Load_MovieDB')
mainDf = PD.DataFrame(mainLs)
NT.GetRandom(mainDf, 2)

,id,TmdbId,Title,OriginalTitle,ReleaseDate,Genres,Collection,Synopsis,Crew,Cast,Companies,Countries,Poster,RunTime,Budget,Gross,Score,ScoreVotes
24244,24245,849135,La Ticla,La Ticla,2020-01-01,None,None,"""This 50D Ektachrome reel showcases some of th...",Dan Smeby,None,None,Canada,/hb05ld27RVoXxOWwMnsH4166IMm.jpg,3.0,NaN,NaN,0.0,0
24245,24246,848758,Tender,Тендер,2020-01-01,"Drama, Comedy, Crime",None,Two former law school classmates decide to go ...,Berik Oshakbaev,"Daniyar Tazhenov, Bekzat Dostanov, Gulnara Sab...",None,Kazakhstan,/rKcKSGkVR9zms2mXVk3VpuLnTEq.jpg,NaN,NaN,NaN,0.0,0


In [11]:
# the non-english characters are preserved
# they aren't displayed properly when opened in excel 

checkDf = mainDf[mainDf['Title']!=mainDf['OriginalTitle']]
NT.GetRandom(checkDf, 4)

,id,TmdbId,Title,OriginalTitle,ReleaseDate,Genres,Collection,Synopsis,Crew,Cast,Companies,Countries,Poster,RunTime,Budget,Gross,Score,ScoreVotes
14844,14845,242293,The Precinct,Sahə,2010-01-01,"Drama, Thriller",None,"Baku, Azerbaijan. A famous photographer postpo...",Ilgar Safat,"Zaza Bejashvili, Melissa Papel, Vaqif Ibrahimo...","Narimanfilm, Georgianfilm, Bagirafilm","Azerbaijan, Georgia, Georgia, Azerbaijan, Georgia",/qd2oSOpGITzUe2ZjklXJvdjG4ll.jpg,116.0,NaN,NaN,5.0,3
14849,14850,238800,The Light,灯花,2010-01-01,Animation,None,A streetlight that breaks and falls to the gro...,Yuta Sukegawa,None,None,Japan,None,7.0,NaN,NaN,0.0,0
14853,14854,232359,See Sing Confirm,4 สิงห์คอนเฟิร์ม,2010-01-06,"Action, Comedy",None,Four people decide to team up to save their ki...,Widit Tanyapan,"Sittichai Pabchompoo, Boriboon Junreurng, Rich...",None,Thailand,None,95.0,NaN,NaN,4.5,1
14863,14864,224064,The Bisexual Revolution,La bisexualité: Tout un art?,2010-01-01,Documentary,None,"Believe it or not, Mick Jagger was not the fir...","Laure Michel, Eric Wastiaux",None,None,France,/ck51GNtHE4E3GhyPWZFWYOL4tJH.jpg,61.0,NaN,NaN,1.0,1
